In [1]:
import json
datas = [json.loads(lines) for lines in open('../data/rotten_tomatoes/arms_MLM_bert_base_uncased_validation_ppls.json','r')]

In [ ]:
data_mlms = [eval(lines) for lines in open('../data/rotten_tomatoes/data_new/arms_MLM_bert_base_uncased_validation.json','r')]
datas = [eval(lines) for lines in open('../data/rotten_tomatoes/arms_MLM_bert_base_uncased_validation_ppls.json','r')]

In [19]:
data_arms = [eval(lines) for lines in open('../data/rotten_tomatoes/data_new/eng_val_new_hash_map.json','r')]

In [23]:
from tqdm import tqdm

def add_ppl_data(data_mlms, sent_id, ppl):
    for _, data in range(len(data_mlms)):
        samples = data['meta']['samples']
        for sample in samples:
            if(sample[0]==sent_id):
                sample.append(ppl)
                with open('../data/rotten_tomatoes/data_new/arms_MLM_bert_base_uncased_validation_ppls_new.json', "a") as fl:
                    json.dump(sample, fl)
                    fl.write("\n")
        
def get_id(word, datas):
    for data in datas:
        word1 = data['meta']['word']
        if(word1==word):
            return data['id']
        
def is_ppl_available(word, sent_id):
    for data in datas:
        if(data['meta']['word']==word):
            samples = data['meta']['samples']
            for sample in samples:
                if(sample[0]==sent_id):
                    return True, data['meta']
                
    return False, []

Start adding


  0%|                                                                                                                                                                                     | 0/5104 [00:00<?, ?it/s]

[0.10775399799835363, 0.10360821129735688, 0.09635507436655324, 0.08678211198900988, 0.10133741156845726, 0.09798545425976084, 0.09127236643387518, 0.13412080922773117, 0.09193603480582473, 0.08884852805307715]
0 0


In [17]:
data_arms = [eval(lines) for lines in open('../data/rotten_tomatoes/data_new/eng_val_new_hash_map.json','r')]

In [18]:
def calculate_perplexity(all_mlm_edges):
    ppls = []
    for edge in all_mlm_edges:
        inputs = tokenizer(edge, return_tensors = "pt")
        inputs = inputs.to(device)
        loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
        ppl = torch.exp(loss).cpu().detach().numpy()
        ppls.append(ppl)

    ppls_inv = [(1/i) for i in ppls]
    res = np.array(ppls_inv)/np.sum(ppls_inv)

    return res

{'id': 1, 'meta': {'word': 'bartlett', 'sentence_ids': [14]}}

In [ ]:
available, not_available = 0, 0
for _, data in enumerate(tqdm(data_arms)):
    idx = data['id']
    word = data['meta']['word']
    sent_ids = data['meta']['sentence_ids']
    
    for sent_id in sent_ids:
        ppl_available, ppl = is_ppl_available(word, sent_id)
        
        if(ppl_available==True):
            available+=1
            d = {}
            d['id'] = idx
            d['meta'] = ppl
            with open('../data/rotten_tomatoes/data_new/arms_MLM_bert_base_uncased_validation_ppls_new.json', "a") as fl:
                json.dump(d, fl)
                fl.write("\n")
                
        elif(ppl_available==False):
            not_available+=1
            for i, sample in enumerate(tqdm(data_mlms)):
                edges = sample['meta']['samples']
                for edge in edges:
                    if(word==sample['meta']['word']):
                        if(edge[0]==sent_id):
                            perplexities = calculate_perplexity(edge[2])
                            edge.append(perplexities.tolist())
                d = {}
                d['id'] = idx
                d['meta'] = sample['meta']
                with open('../data/rotten_tomatoes/data_new/arms_MLM_bert_base_uncased_validation_ppls_new.json', "a") as fl:
                    json.dump(d, fl)
                    fl.write("\n")
            

NameError: name 'NaN' is not defined

In [3]:
import json
from tqdm import tqdm

reward_fpath = "../data/rotten_tomatoes/data_new/arms_MLM_bert_base_uncased_validation_ppls.json"
#data_reward = [eval(lines) for lines in open(reward_fpath, "r")]

f = open(reward_fpath,'r')
for line in tqdm(f):
    try:
        lin = eval(line)
        with open('../data/rotten_tomatoes/data_new/arms_MLM_bert_base_uncased_validation_ppls_correct.json','a') as f1:
            json.dump(lin, f1)
            f1.write('\n')
    except:
        #print(line['id'])
        continue

5104it [00:39, 128.07it/s] 


In [4]:
reward_fpath = "../data/rotten_tomatoes/data_new/arms_MLM_bert_base_uncased_validation_ppls_correct.json"
data_reward = [eval(lines) for lines in open(reward_fpath, "r")]

In [8]:
x = [1,1,0,1,1,1]

In [11]:
if ~(True) in x:
    print('S')

In [6]:
import pandas as pd
import json
from tqdm import tqdm

data = json.load(open('../data/rotten_tomatoes/data_new/all_words_flips_0.9.json','r'))
df = pd.read_csv('../data/rotten_tomatoes/data_test/generated_text_0308.csv')

def calculate_SSR(data, df):
    word_level_SSR_cnt = 0
    sent_level_SSR_cnt = [False]*len(df)
    for dat in tqdm(data):
        flag=False
        for sample in dat['meta']:
            if(sample['flip']==True):
#                 if(sent_level_SSR_cnt[sample['idx']]==False):
#                     sent_level_SSR_cnt[sample['idx']]=True                 
                flag=True
                print(dat['word'])
                break
        if(flag==True):
            word_level_SSR_cnt+=1
    print(word_level_SSR_cnt)
    word_level_SSR = word_level_SSR_cnt/len(data)
    #sent_level_SSR = sum(sent_level_SSR_cnt)/len(df)
    print(f'Word level SSR: {word_level_SSR}')
    #print(f'Sentence level SSR: {sent_level_SSR}')
    
calculate_SSR(data, df)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 86153.27it/s]

aims
weighs
mongrel
stirs
average
seeing
6
Word level SSR: 0.3157894736842105


In [4]:
len(data)

19

In [5]:
word_level_SSR_cnt

NameError: name 'word_level_SSR_cnt' is not defined

In [26]:
len(data)

414

In [31]:
data = json.load(open('../data/rotten_tomatoes/data_new/all_words_flips.json','r'))
len(data)

948

In [13]:
import json

employees_string = '''
{"span": "Twilight", "category": "Material", "foreignness": "Generally Recognizable", "factors": "Country, Age, Genre preference", "explanation": "Fantasy genre popular in USA among teenagers."},
{"span": "vampires", "category": "Material", "foreignness": "Generally Recognizable", "factors": "Country, Age, Genre preference", "explanation": "Fantasy genre popular in USA among teenagers."},
{"span": "romance", "category": "Customs", "foreignness": "Generally Recognizable", "factors": "Country, Age, Genre preference", "explanation": "Romance novels popular among teenagers in USA."},
{"span": "teenage girl", "category": "Social", "foreignness": "Generally Recognizable", "factors": "Country, Age, Genre preference", "explanation": "Teenage girl culture in USA."}

'''

data = json.loads(employees_string)

print(type(data))
#output
#<class 'dict'>

#access first_name
# for employee in data["employees"]: 
#     print(employee["first_name"])

JSONDecodeError: Extra data: line 2 column 195 (char 195)

In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent, ent.label_)

Apple ORG
U.K. GPE
$1 billion MONEY


In [1]:
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_sm")
from tqdm import tqdm

In [2]:
df = pd.read_csv('../data/checklist/checklist_changelocations_INV.csv')

In [35]:
words = []
for idx, row in tqdm(df.iterrows()):
    text = row['text']
    doc = nlp(text)
    for ent in doc.ents:
        if(ent.label_ == 'GPE'):
            if ent.text not in words:
                words.append(ent)
print(words)

9999it [00:52, 192.24it/s]

[Manchester, Newark, Hartford, Newark, Sparks, Newark, Manchester, Manchester, Newark, Manchester, Manchester, St. Cloud, Newark, Boston, Lakeville, Carlsbad, Kansas City, Jefferson City, Galveston, Palm Springs, South San Francisco, Boston, Bridgeport, Binghamton, Yakima, Syracuse, Mishawaka, Lawton, Margate, Denver, Trenton, Brooklyn Park, Salem, Fresno, Modesto, Trenton, Denver, Rochester, Fort Collins, San Buenaventura, Ventura, South Jordan, Portland, Fort Collins, Garland, Brookfield, Tucson, Vegas, Jacksonville, Vegas, Rock Hill, Vegas, Vegas, Vegas, Vegas, Lewisville, Vegas, Vegas, Hammond, Vegas, Vegas, Vegas, Houston, Newark, Harrisburg, Newark, Delray Beach, Newark, Houston, Linden, Houston, Houston, Hampton, Newark, Houston, Orlando, Newark, Salem, Newark, Houston, Yuma, Columbus, Oakland, Columbus, Oakland Park, Columbus, Columbus, Columbus, Columbus, Columbus, Columbus, Columbus, Columbus, Columbus, Korea, Chicago, Estonia, Chicago, Colombia, Chicago, Hungary, Chicago, Ko

In [34]:
len(words)

9502

In [6]:
import numpy as np

def unique(list1):
    x = np.array(list1)
    return np.unique(x)
    
unique_words = unique(words)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (9502,) + inhomogeneous part.

In [8]:
words1 = ["Manchester", "Newark", "Hartford", "Sparks", "St. Cloud", "Boston", "Lakeville", "Carlsbad", "Kansas City", "Jefferson City", "South San Francisco", "Boston", "Bridgeport", "Binghamton", "Yakima", "Syracuse", "Mishawaka"]

In [50]:
sens = [eval(line) for line in open('../experiments_new/thompson_01_eg_binary_cl_500_200k/global_sensitivity_finalvalues.json','r')]

In [56]:
for sen in sens:
    for key, value in sen.items():
        if value<0.2:
            print(key)

#daddyshome
belabor
#delay
#ohio
scratching
screens
announces
stall
fred
ship
sitters
muscles
frown
flyin
dating
nov
greg
cluster
encounter
hockey
matches
exhausted
lighting
windows
larger
january
cozy
#ohare
#gross
#helpmeplease
#anyonethere
um
idiots
forgotten
mh
clarify
practice
replacement
tend
#atl
forecasted
facebook
moveup
relatives
hubby
skin
#noluv
enroute
reachable
encourage
sole
stable
#imaginedragons
#slaycancerwithdragons
grabbed
daytona
protect
srvc
spends
stranger
#randomactsofcorporategreed
espinosa
disconnect
booo
aunts
prompting
backs
wounded
vet
healing
pillows
blanket
functioning
lining
ers
curtesy
sched
venezuela
lift
themed
dishonest
#faa
responds
#wedontcarebecauseyoupaidalready
shenanigans
movies
#wronganswer
recognition
#missedupgrades
resolutions
mn
mammoth
#makingthingseasy
statement
telephone
comment
static
learning
marketing
teenage
ha
crosswinds
#comeonpeople
#lovejetblue
terminals
technical
illiterate
cst
jacked
lowered
marsh
thnx
discounts
sometime
#yuck

In [32]:
for key, value in sens[0].items():
    if key in words1:
        print(f'{key} : {value}')

switzerland : 0.13188160468002963
#portland : 0.15484165272867764
#vancouver : 0.0986354966278242
#danville : 0.013412432742123559
#turkmenistan : 0.07818762960492626
bolivia : 0.17494786746440869
pakistan : 0.1473474987302995
indonesia : 0.1956751926798888
somalia : 0.7223421486440238
grenada : 0.08351941031578067
belgium : 0.1401329154393458
bulgaria : 0.028648081283963456
azerbaijan : 0.5504883693535451


In [20]:
len(sens[0]["Guinea"])

KeyError: 'Guinea'

In [21]:
sens

[{'#kearny': 0.21325553982046214,
  '#sherman': 0.1708952058021019,
  '#renton': 0.1061771043968334,
  '#victoria': 0.08737989412105723,
  '#carson': 0.18643013146885778,
  '#lynchburg': 0.16945738537599273,
  '#abilene': 0.2300655211155907,
  '#odessa': 0.11162971032398325,
  '#houston': 0.1336285332848434,
  '#frederick': 0.025155633323627524,
  '#compton': 0.2492168175346751,
  '#bridgeport': 0.21349275468357637,
  '#baytown': 0.08634856063027535,
  '#beaumont': 0.10087002425281724,
  '#crystal': 0.06454210909174302,
  '#baldwin': 0.15254488240572142,
  '#burlington': 0.13098704547933454,
  '#clovis': 0.04790885732700917,
  '#ankeny': 0.21126335904250537,
  '#lincoln': 0.08833175140428243,
  '#tinley': 0.13789142278665817,
  '#apex': 0.2178861376239859,
  '#rancho': 0.2475915283975552,
  'uruguay': 0.055416752783857715,
  '#troy': 0.11289189752006584,
  '#danbury': 0.11658888878767132,
  '#torrance': 0.019446807562195134,
  "#o'fallon": 0.1318134514225109,
  '#kingsport': 0.17437271

In [36]:
pwd

'/home/debrupdas/MAB_SKP/SMAB/SMAB/MAB/utils'

In [38]:
df = pd.read_csv('../data/checklist/checklist_all_types_500/temp_preds.csv')
df

,arm,example_id,new_word,mlm_score,text,bool_original,prediction,probabilities
0,greyed,2082,way,0.127664,How do I reschedule for Cancelled Flightled fl...,0,0,"[0.7570163011550903, 0.23242279887199402, 0.01..."
1,greyed,2082,timed,0.100084,How do I reschedule for Cancelled Flightled fl...,0,0,"[0.668324887752533, 0.32040730118751526, 0.011..."
2,greyed,2082,always,0.065793,How do I reschedule for Cancelled Flightled fl...,0,0,"[0.732841432094574, 0.2569899559020996, 0.0101..."
3,greyed,2082,already,0.050289,How do I reschedule for Cancelled Flightled fl...,0,0,"[0.656783938407898, 0.3322524130344391, 0.0109..."
4,greyed,2082,checked,0.043744,How do I reschedule for Cancelled Flightled fl...,0,1,"[0.3245867192745209, 0.6560604572296143, 0.019..."
...,...,...,...,...,...,...,...,...
74845,think,16696,know,0.001294,do i know that company is dreadful? no.,0,0,"[0.8621717095375061, 0.1225784569978714, 0.015..."
74846,think,16696,mean,0.000840,do i mean that company is dreadful? no.,0,0,"[0.9071752429008484, 0.08438100665807724, 0.00..."
74847,think,16696,find,0.000671,do i find that company is dreadful? no.,0,0,"[0.8671878576278687, 0.11964993923902512, 0.01..."
74848,think,16696,suppose,0.000489,do i suppose that company is dreadful? no.,0,0,"[0.8851727843284607, 0.10601577162742615, 0.00..."


In [48]:
def csv_to_json(ip_json_file, op_json_file, csv_file):
    
    data_reward = [eval(lines) for lines in open(ip_json_file, "r")]
    df = pd.read_csv(csv_file)
    new_arms = []
    
    with open(op_json_file, 'w') as file:
        
        for i in tqdm(range(len(data_reward))):

            try:
                arm = data_reward[i]['meta']['word']

                # Samples is the list of edges
                samples = data_reward[i]['meta']['samples'][0]
                example_no = samples[0]
                replacement_words = samples[1]
                sentences = samples[2]
                mlm_scores = samples[3]

                new_samples = []

                # Find in the dataframe df with arm - multiple edges 
                armmatch_df = df[(df['arm'] == arm)]


                if len(armmatch_df) == 0:
                    new_arms.append(data_reward[i]) # Keep as before 
                    continue

                armmatch_df = armmatch_df.reset_index(drop=True)

                #print("Before Adding column",armmatch_df)
                probs = armmatch_df['probabilities'].tolist()
                preds = armmatch_df["prediction"].tolist()

                new_probs = []
                new_preds = []
                bool_original = armmatch_df["bool_original"].tolist()

                #print(arm)
                for j in range(len(replacement_words)):
                    index_of_value = armmatch_df.index[armmatch_df['new_word'] == replacement_words[j]].tolist()
                    #print(preds, index_of_value)
                    new_preds.append(preds[index_of_value[0]])
                    new_probs.append(probs[index_of_value[0]])

                orig_ind = (armmatch_df.index[armmatch_df['bool_original'] == 1].tolist())[0]
                orig_pred = preds[orig_ind]
                orig_prob = probs[orig_ind]

                pred_list = [orig_pred]
                pred_list.append(new_preds)

                prob_list = [orig_prob]
                prob_list.append(new_probs)

                samples.append(pred_list) 
                samples.append(prob_list)

                data_reward[i]['meta']['samples'] = [samples]
                new_arms.append(data_reward[i])
                s = str(data_reward[i])
                file.write(s)
                file.write('\n')
            except:
                continue

In [49]:
# Convert csv file to json 
folder_path = "../data/checklist/checklist_all_types_500"
csv_to_json(ip_json_file=f"{folder_path}/arms_MLM_roberta_base_uncased_validation.json",
           op_json_file=f"{folder_path}/arms_MLM_roberta_base_uncased_validation_preds.json",
            csv_file=f"{folder_path}/temp_preds.csv")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6807/6807 [00:25<00:00, 264.08it/s]


In [61]:
!pip install lime